In [1]:
import glob,os
import spacy
import json
from pathlib import Path
import re

In [2]:
#nlp = spacy.load('en_core_web_sm')

In [3]:
path = r"C:\Users\acer\Downloads\MEDiMakers test-buddy main data-page_texts_2_rikhil"

In [4]:
file_paths = []
for f in os.listdir(path):
    file_paths.append((os.path.join(path,f)))

In [148]:
for file in file_paths:
    name = Path(file).stem 
    figs = []
    if name.endswith(("17","30")):
        continue
    with open(file,'r',encoding='utf-8') as f:
        text = "".join((f.readlines()))
        parsed_txt = (list(filter(lambda x:len(x)>5,text.split(". "))))
        cleaned_txt = (".\n".join(parsed_txt))
        figs+=(re.findall("Fig.*? \d-\d*",cleaned_txt)) # replace instances of figures w just the number in a N.NN format
        figs+=(re.findall("\(S.* figure.*?\d-\d*\)|\(see figure.*?\d-\d*\)",cleaned_txt)) # remove references to figures >> rmv (see fig ...)
        for k in figs:
            if k.startswith("Fig"):
                n = re.findall("\d+",k)
                cleaned_txt = cleaned_txt.replace(k,f"{n[0]}.{n[1]}")
            else:
                cleaned_txt = cleaned_txt.replace(k,"")
        if "&" in cleaned_txt:
            cleaned_txt =  cleaned_txt.replace("&",'and') # replace ampersand with &
        if len(re.findall("\so\s",cleaned_txt))>0:
            cleaned_txt = (re.sub(r"(\s)o(\s)",r"\1-\2",cleaned_txt)) # standardise bullet points (o) with hypen (-)

        cleaned_txt = (re.sub("\uf0a9","-",cleaned_txt)) # change non-unicode recognised bulletpoints w hyphen
    with open(file,'w',encoding='utf-8') as f: # write to same file
        f.write(cleaned_txt)    # manual checks performed o this stage
        #sentences = [i.text for i in nlp(text).sents] 
        #print("\n".join(sentences))
        #data[name] = info 
    #with open(f"{name}.json",'w',encoding='utf-8') as g:
        #json.dump(data,g,ensure_ascii=False)

In [160]:
# dump json info after consolidated
for file in file_paths:
    name = Path(file).stem 
    pg = (name.split("_")[-1])
    data = {name:{}}
    info = {}
    if name.endswith(("17","30")):
        continue
    with open(file,'r',encoding='utf-8') as f:
        content = ("".join(f.readlines()))
        data[name]["Text"] = content 
        data[name]["Page"] = str(pg) 
    with open(f"{name}.json",'w',encoding='utf-8') as j:
        json.dump(data,j,ensure_ascii=False,indent=4)